In [ ]:
# Redmine ticket 6870 - Siblings auto-complete, for all visits completed.
# Imports
import json
import os 
import pandas as pd
from django.apps import apps as django_apps
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [ ]:
from copy import copy
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT, INCOMPLETE_APPT
from edc_metadata.constants import REQUIRED, KEYED

sidx = ['B142-040990713-5', 'B142-040990466-0', 'B142-040990612-9', 'B142-040990837-2', 'B142-040990857-0',
        'B142-040990022-1', 'B142-040990666-5', 'B142-040990562-6', 'B142-040990705-1', 'B142-040990570-9',
        'B142-040990804-2', 'B142-040990786-1', 'B142-040990558-4', 'B142-040990023-9', 'B142-040990355-5',
        'B142-040990669-9', 'B142-040990099-9', 'B142-040990528-7', 'B142-040990172-4', 'B142-040991009-7',
        'B142-040990044-5', 'B142-040990116-1', 'B142-040990639-2', 'B142-040990120-3', 'B142-040990893-5']

for sid in sidx:
    completed_quart_appts = Appointment.objects.filter(
        subject_identifier=sid, schedule_name__icontains='quart').exclude(appt_status=NEW_APPT)
    for appt in completed_quart_appts:
        try:
            completed_visit = MaternalVisit.objects.get(appointment=appt)
        except MaternalVisit.DoesNotExist:
            continue
        else:
            new_appt = Appointment.objects.filter(
                subject_identifier=sid, visit_code=appt.visit_code,
                visit_code_sequence=appt.visit_code_sequence).exclude(pk=appt.pk)
            if new_appt.exists():
                new_appt = new_appt[0]
                try:
                    new_visit = MaternalVisit.objects.get(appointment=new_appt)
                except MaternalVisit.DoesNotExist:
                    new_visit = copy(completed_visit)
                    new_visit.id = None
                    new_visit.schedule_name = new_appt.schedule_name
                    new_visit.visit_schedule_name = new_appt.visit_schedule_name
                    new_visit.appointment = new_appt             
                    new_visit.save()

                    crf_metadata = CrfMetadata.objects.filter(
                        subject_identifier=sid, visit_code=new_visit.visit_code,
                        visit_code_sequence=new_visit.visit_code_sequence,
                        schedule_name=new_visit.schedule_name, )

                    crf_metadata.delete()
                    metadata = new_visit.metadata_cls(visit=new_visit, update_keyed=False)
                    metadata.prepare()
    
                    new_visit.save()
                else:
                    crf_metadata = CrfMetadata.objects.filter(
                        subject_identifier=sid, visit_code=new_visit.visit_code,
                        visit_code_sequence=new_visit.visit_code_sequence,
                        schedule_name=new_visit.schedule_name, )

                    crf_metadata.delete()
                    metadata = new_visit.metadata_cls(visit=new_visit, update_keyed=False)
                    metadata.prepare()
    
                    new_visit.save()

                # change new appointment status to incomplete
                new_appt.appt_datetime = appt.appt_datetime
                new_appt.appt_status = INCOMPLETE_APPT
                new_appt.appt_reason = appt.appt_reason
                new_appt.save()
